In [6]:
from core.scene_detection import detect_scene_changes

# Path to the input video file
video_path = 'test_files/big_file.mov'

print(f"Detecting scene changes in '{video_path}'...")
# Call the function to detect scene changes
frame_numbers = detect_scene_changes(video_path)

print(f"Finished detecting {len(frame_numbers)} scenes in '{video_path}'.")

print(frame_numbers)


VideoManager is deprecated and will be removed.


Detecting scene changes in 'test_files/big_file.mov'...
Finished detecting 213 scenes in 'test_files/big_file.mov'.
[0, 685, 1245, 1353, 1383, 1572, 1746, 1771, 1817, 2508, 2925, 3046, 3094, 4364, 4537, 4656, 4715, 4803, 5125, 5501, 6136, 6332, 6468, 6852, 6928, 7070, 7188, 7279, 7508, 8013, 8126, 8353, 8557, 8925, 9069, 9291, 9888, 9981, 10031, 10182, 10378, 10853, 11177, 11423, 11608, 11681, 11846, 12324, 12548, 12861, 12905, 12969, 13024, 13147, 13267, 13420, 13529, 13555, 13610, 13692, 13759, 13806, 13879, 14039, 14059, 14315, 14403, 14474, 14603, 14815, 15087, 15248, 15405, 16006, 16532, 17010, 17116, 17159, 17199, 17475, 17625, 18113, 18738, 18971, 19198, 19348, 19443, 19641, 19788, 19918, 19941, 20061, 20869, 20988, 21046, 21120, 21176, 21478, 21557, 21608, 21665, 21722, 22314, 22393, 22550, 22672, 22764, 23312, 23472, 23711, 23834, 23950, 24075, 24200, 24493, 24802, 24973, 25050, 25088, 25508, 25569, 25731, 25818, 25908, 26033, 26131, 26333, 26579, 26891, 26951, 27025, 27532, 2

In [7]:
# Output the frame_numbers array into a file
output_file = 'frame_numbers_big_file.txt'
with open(output_file, 'w') as f:
    for frame in frame_numbers:
        f.write(f"{frame}\n")

In [7]:
# Read the numbers from 'last_frames.txt' and load them into an array
input_file = 'frame_numbers2.txt'
loaded_frame_numbers = []

with open(input_file, 'r') as f:
    for line in f:
        loaded_frame_numbers.append(int(line.strip()))

print(loaded_frame_numbers)

[0, 300, 533, 1121, 1778]


In [8]:
import mido
from mido import MetaMessage, MidiFile, MidiTrack


tpr = 960 / 25

adjusted_frame_numbers = [round(frame * tpr) for frame in loaded_frame_numbers]

# Create a new MIDI file and a single track
mid = MidiFile(ticks_per_beat=960)
track = MidiTrack()
mid.tracks.append(track)

# Convert BPM to tempo
tempo = mido.bpm2tempo(60)

# Add tempo and time signature to the track
track.append(MetaMessage('set_tempo', tempo=tempo))
track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

# Add markers at each scene change frame number
previous_tick = 0
for i, frame in enumerate(adjusted_frame_numbers):
    # Calculate the tick difference from the previous frame
    if i == 0:
        tick_diff = 0
    else:
        tick_diff = frame - adjusted_frame_numbers[i - 1]
    
    marker_text = f'SC {i + 1}'  # Custom marker text for each scene
    
    # Add a marker (meta event) at the scene change time
    track.append(MetaMessage('marker', text=marker_text, time=tick_diff))
    previous_tick += tick_diff

# Save the MIDI file with the provided name
mid.save("test_output23.mid")
print(f"MIDI file magla_round_precise.mid created successfully.")

MIDI file magla_round_precise.mid created successfully.


In [3]:
import cv2

def get_video_fps(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Get the FPS (frames per second) of the video
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Release the video capture object
    video_capture.release()
    
    return fps

# Example usage
video_path = 'test_files/big_file.mov'
fps = get_video_fps(video_path)
print(f"FPS of the video: {fps}")

FPS of the video: 24.0
